**Creating Gold Schema**

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ecommerce.gold_olist;


**Gold Analytics Table (Business KPIs)
**

Purpose

- This table answers business questions, not ML:
- Total orders
- Delayed orders
- Delay percentage
- Average delay days

This satisfies:
- Analytics
- Business impact
-Insight generation

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold_olist.delivery_kpis AS
SELECT
    COUNT(*) AS total_orders,
    SUM(is_delayed) AS delayed_orders,
    ROUND(SUM(is_delayed) * 100.0 / COUNT(*), 2) AS delayed_percentage,
    ROUND(AVG(delivery_delay_days), 2) AS avg_delay_days
FROM ecommerce.silver_olist.orders_delivery_status;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM ecommerce.gold_olist.delivery_kpis;


total_orders,delayed_orders,delayed_percentage,avg_delay_days
99441,6535,6.57,-11.88


**Gold ML Feature Table (Model Input)**

Purpose
- This is the final dataset used for ML.
- Why this is Gold (not Silver):
- Feature selection is final
- No more transformations
- Direct ML consumption

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold_olist.ml_features AS
SELECT
    delivery_delay_days,
    is_delayed
FROM ecommerce.silver_olist.orders_delivery_status
WHERE order_delivered_customer_ts IS NOT NULL;


num_affected_rows,num_inserted_rows


This table:
- Has target variable
- Has numeric feature
- Is clean and ML-ready

Now, I will verify this, If count > 0 then it is correct and verified.

In [0]:
%sql
SELECT COUNT(*) FROM ecommerce.gold_olist.ml_features;


COUNT(*)
96476


Confirming Gold Completion

it should display: delivery_kpis & ml_features

In [0]:
%sql
SHOW TABLES IN ecommerce.gold_olist;


database,tableName,isTemporary
gold_olist,delivery_kpis,false
gold_olist,ml_features,false


Finally, Gold analytics and ML feature tables have been created.

**ML model + MLflow**

ML Objective
Problem type: Binary Classification
Question: Will an order be delayed (is_delayed = 1) or not?

Target column: "is_delayed"
Feature(s): delivery_delay_days

Loading ML dataset from Gold table

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [0]:
df = spark.table("ecommerce.gold_olist.ml_features").toPandas()

X = df[["delivery_delay_days"]]
y = df["is_delayed"]


Train/Test split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Train model + track with MLflow

In [0]:
with mlflow.start_run(run_name="logistic_reg_delivery_delay"):

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)

    mlflow.sklearn.log_model(model, "delivery_delay_model")


2026/02/01 17:40:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ This satisfies:
- Model training
- Evaluation
- MLflow tracking
- Explainability

Store prediction back to Gold (This is very important)
Creating prediction dataframe

In [0]:
df["prediction"] = model.predict(X)

predictions_df = spark.createDataFrame(df)


Write prediction table

In [0]:
predictions_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce.gold_olist.delivery_predictions")


now verifying

In [0]:
%sql
SELECT COUNT(*) FROM ecommerce.gold_olist.delivery_predictions;


COUNT(*)
96476


✔ ML model trained
✔ Metrics logged in MLflow
✔ Predictions stored in Lakehouse
✔ End-to-end Database ↔ AI workflow

**Delta Lake Optimization**
This explicitly satisfies: Delta Lake: Tables with ACID transactions and optimisation

I will optimize: Only Gold tables:
- ecommerce.gold_olist.delivery_kpis
- ecommerce.gold_olist.ml_features
- ecommerce.gold_olist.delivery_predictions

In [0]:
%sql
OPTIMIZE ecommerce.gold_olist.ml_features;


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1769968249388, 1769968250158, 8, 0, null, List(0, 0), null, 2, 2, 0, 0, null)"


In [0]:
%sql
OPTIMIZE ecommerce.gold_olist.delivery_predictions;


path,metrics
,"List(1, 8, List(79586, 79586, 79586.0, 1, 79586), List(12919, 13159, 13093.125, 8, 104745), 0, null, null, 0, 1, 8, 0, true, 0, 0, 1769968263761, 1769968266227, 8, 1, null, List(0, 0), null, 3, 3, 474, 0, null)"


In [0]:
%sql
OPTIMIZE ecommerce.gold_olist.delivery_kpis;


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1769968274727, 1769968275305, 8, 0, null, List(0, 0), null, 4, 4, 0, 0, null)"
